In [4]:
import json
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

f = open('final-results.json', )
data = json.load(f)
f.close()


configs = []
for x in data['configs']:
    configs.append({
        'fitness_function': x['evolution']['fitness_function'],
        'survival_criterion': x['evolution']['survival_selection_criterion'],
        'reproduction_criterion': x['evolution']['reproduction_selection_criterion'],
        'mutation_rate': x['evolution']['mutation_rate'],
        'survival_rate': x['evolution']['survival_rate'],
        'max': x['stats']['time_steps'],
        'mean': np.mean(x['stats']['time_step_history']),
        'median': np.median(x['stats']['time_step_history']),
        'step_history': x['stats']['time_step_history'],
        'best_ca': x['stats']['best_individual'],
        'best_generation': x['stats']['best_generation'],
    })


df = pd.DataFrame(configs)
df = pd.concat([df.drop(['best_ca'], axis=1), df['best_ca'].apply(pd.Series)], axis=1)
df['successful'] = df['max'] > 15000
successful = df[df['successful'] == True]

In [5]:
df[['fitness_function', 'survival_criterion', 'reproduction_criterion', 'mutation_rate', 'survival_rate', 'max', 'best_generation']]

,fitness_function,survival_criterion,reproduction_criterion,mutation_rate,survival_rate,max,best_generation
0,position_based,rank_proportional,elitist,0.010,0.1,4295.0,28
1,angle_and_time_based,rank_proportional,uniform,0.010,0.0,955.0,10
2,total_time_steps,uniform,elitist,0.010,0.0,1917.0,18
3,total_time_steps,uniform,elitist,0.010,0.1,871.0,12
4,position_based,uniform,fitness_proportional,0.010,0.1,1437.0,24
...,...,...,...,...,...,...,...
94,time_based,fitness_proportional,tournament,0.001,0.1,12105.0,17
95,position_and_angle_based,elitist,tournament,0.001,0.1,3439.0,26
96,position_based,fitness_proportional,rank_proportional,0.001,0.2,1797.0,3
97,angle_and_time_based,uniform,rank_proportional,0.001,0.0,567.0,26


In [6]:
ax = sns.histplot(data=df, x="max")
ax.set_xlabel('Max time steps')
ax.set_ylabel('Count')
plt.savefig('max_dist.png')

AttributeError: module 'seaborn' has no attribute 'histplot'

In [7]:
ax = sns.histplot(data=df, x="mean")
ax.set_xlabel('Mean time steps')
ax.set_ylabel('Count')
plt.savefig('mean_dist.png')

AttributeError: module 'seaborn' has no attribute 'histplot'

In [ ]:
ax = sns.histplot(data=df, x="median")
ax.set_xlabel('Median time steps')
ax.set_ylabel('Count')
plt.savefig('median_dist.png')

In [ ]:
ax = sns.histplot(data=df, x="best_generation")
ax.set_xlabel('Generation #')
ax.set_ylabel('Count')
plt.savefig('gen_dist.png')

In [ ]:
df['mean'].mean()

In [ ]:
df['median'].median()

In [ ]:
len(df), len(df[df['max'] > 1000])

In [ ]:
len(df), len(df[df['max'] > 10000])

In [ ]:
len(df), len(df[df['max'] > 20000])

In [ ]:
len(df), len(df[df['max'] > 25000])

In [ ]:
maxed_out = df[df['max'] == 29999]
len(maxed_out)

In [ ]:
len(successful)

In [ ]:
successful['mean'].mean(), successful['median'].median()

In [ ]:
df.groupby('fitness_function')['max'].max()

In [ ]:
df.groupby('fitness_function')['mean'].mean()

In [ ]:
df.groupby('fitness_function')['median'].median()

In [ ]:
successful.groupby('fitness_function')['successful'].count().sort_values(ascending=False)

In [ ]:
df['fitness_function_short'] = df['fitness_function'].replace(
    {
        'angle_based': 'ab',
        'position_based': 'pb',
        'angle_and_time_based': 'atb',
        'time_based': 'tb',
        'total_time_steps': 'tts',
        'position_and_angle_based': 'pab',
    }
)

df

In [ ]:
from lets_plot import *
ggplot(df) + geom_point(aes(x="action_index", y="best_generation")) + ggtitle("Hello!") + xlab("Some Axi label")

In [ ]:
data = df.explode('step_history').reset_index(drop=True)
data.step_history = data.step_history.astype(float)

In [ ]:

#sns.set(rc={'figure.figsize':(11.7,8.27)})
ax = sns.violinplot(x="fitness_function_short", y="mean", data=df)
ax.set_xlabel('Fitness function')
ax.set_ylabel('Count')
plt.savefig('mean_violin.png')

In [ ]:

#sns.set(rc={'figure.figsize':(11.7,8.27)})
ax = sns.violinplot(x="fitness_function_short", y="median", data=df)
ax.set_xlabel('Fitness function')
ax.set_ylabel('Count')
plt.savefig('median_violin.png')

In [ ]:
successful.fitness_function.value_counts()

In [ ]:
successful.survival_criterion.value_counts()

In [ ]:
successful.reproduction_criterion.value_counts()

In [ ]:
top_10 = successful.sort_values(by='max', ascending=False).head(10)

In [ ]:
fig = go.Figure()

for x in range(len(top_10)):
    values = top_10[x:x+1].step_history.values.tolist()

    out = []
    for idx, y in enumerate(values[0]):
        out.append({'value': y})

    data = pd.DataFrame(out)

    fig.add_trace(go.Scatter(y=data['value'],
                             mode='lines',
                             name=x +1 ))
    continue
fig.show()

In [ ]:
df.groupby(['fitness_function', 'survival_criterion', 'reproduction_criterion'])['mean'].max().reset_index().sort_values(by='mean',ascending=False).head(5)

In [ ]:
df.groupby(['fitness_function', 'survival_criterion', 'reproduction_criterion'])['median'].max().reset_index().sort_values(by='median',ascending=False).head(5)

In [ ]:
df['pole_angle'] = df['pole_angle'].apply(lambda x: x[0])
df = pd.concat([df.drop(['pole_angle'], axis=1), df['pole_angle'].apply(pd.Series)], axis=1)
df['pole_angle'] = df['value']

In [ ]:
successful.rule_number.value_counts().head(5)

In [ ]:
successful.survival_rate.value_counts()

In [ ]:
ax = sns.histplot(data=df, x="pole_angle", hue='successful')
ax.set_xlabel('Angle value')
ax.set_ylabel('Count')
plt.savefig('angle_dist.png')